In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import csv

In [2]:
ta = "https://www.tripadvisor.com"

In [3]:
pg_nos = [ "", "oa30-","oa60-"]

In [4]:
hotel_list = []
hotel_links = []

In [5]:
for pg in pg_nos:
    r = requests.get("https://www.tripadvisor.com/Hotels-g60745-"+ str(pg) +"Boston_Massachusetts-Hotels.html")
    soup = BeautifulSoup(r.content,"lxml")
    hotels = soup.find_all("div", {"class" : "listing easyClear p13n_imperfect "})
    for hotel in hotels:
        for h in (hotel.contents[1].find_all("a",{"class":"property_title"})):
            hotel_list.append(h.text)
            hotel_links.append( ta + h.attrs["href"])           

In [6]:
rating_numbers = [5,4,3,2,1]

# Traveller Ratings

In [7]:
header = ("hotel_name", "rating", "count")
f = open('traverler_ratings.csv',"w", newline='\n' )
tr_writer = csv.writer(f)
tr_writer.writerow(header)

25

In [8]:
for index, link in enumerate(hotel_links):
    
    r = requests.get(link)
    soup = BeautifulSoup(r.content,"lxml")
    ratings = soup.find_all("div", {"id" : "ratingFilter"})
    for h in ratings:
        for n in rating_numbers:
            for i in (h.contents[3].find_all("label",{"for":"taplc_prodp13n_hr_sur_review_filter_controls_0_filterRating_"+str(n)})):
                a =  (" ".join((i.text).split())).rsplit(' ',1)
                tr_writer.writerow((hotel_list[index], a[0], a[1]))

In [9]:
f.close()

# Attribute Rating

In [10]:
new_heading = ("hotel_name", "review_id", "attribute", "star_value")    # Creates the CSV file
fi = open('attribute_ratings.csv',"w", newline='\n' )
new_writer = csv.writer(fi)
new_writer.writerow(new_heading)

43

In [11]:
def page_numbers( url ):   # To calculate all the page numbers of the reviews for a given hotel
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"lxml")
    a = (soup.find_all("div", {"class":"pageNumbers"}))
    for i in a:
        b = (i.text) 
    x = re.sub('\n',"-",b)
    return (int(x.split("-")[-2]))

In [12]:
def page_reviews(page_url,h):   # Creates the url for each review in a page and sends it to ratings_review function
    r = requests.get(page_url)
    soup = BeautifulSoup(r.content,"lxml")
    review = soup.find_all("div", {"class":"reviewSelector"})
    for stuff in review:
        review_id = stuff.attrs["id"][7:]
        rn = "rn" + review_id
        for rh in stuff.find_all("a",{"id":rn}):
            ratings_review(ta + rh.attrs["href"],review_id,h) 

In [13]:
def ratings_review(url,review_id,h):  # Looks at each review 
    global  writer                # Glabal list for all hotel reviews and attribute stars
    rr = requests.get(url)
    soupp = BeautifulSoup(rr.content,"lxml")
    rev =  "review_" + str(review_id)
    revieww = soupp.find_all("div", {"id":rev})   
    values,keys = [], []
    for i in revieww:
        for j in (i.find_all("span",{"class":"rate sprite-rating_ss rating_ss"})):
            values.append(j.contents[1].attrs["alt"][0])     # Gets the stars for each attribute of a review
        for j in (i.find_all("div",{"class":"recommend-description"})):   
            keys.append((j.contents)[0])                     # Gets the attribute name
        a = list(zip(keys,values))
        if a != []:                                         # Checks if rating has an attribute
            for i in a:
                new_writer.writerow((h,review_id,i[0],i[1]))

In [14]:
for i in range(len(hotel_links)):        # calling the above functions on each hotel
    print(i, time.asctime(), hotel_list[i] )
    hotel = hotel_list[i]
    hlink = hotel_links[i]
    str1 = hlink
    str2 = "Reviews"
    ind = str1.index(str2) + len(str2)
    for i in ( range( page_numbers(hlink) ) ):
         page_reviews(hlink[:ind]+str("-or")+str(i*10) + hlink[ind:] ,str(hotel))

0 Fri Dec 30 18:18:47 2016 Marriott Vacation Club Pulse at Custom House, Boston
1 Fri Dec 30 18:32:08 2016 Seaport Boston Hotel
2 Fri Dec 30 19:42:54 2016 Four Seasons Hotel Boston
3 Fri Dec 30 20:08:20 2016 Boston Harbor Hotel
4 Fri Dec 30 20:37:14 2016 Lenox Hotel
5 Fri Dec 30 21:28:54 2016 Courtyard Boston Copley Square
6 Fri Dec 30 21:54:40 2016 InterContinental Boston
7 Fri Dec 30 22:52:39 2016 Mandarin Oriental, Boston
8 Fri Dec 30 23:02:46 2016 Hotel Commonwealth
9 Sat Dec 31 00:09:55 2016 Residence Inn by Marriott Boston Back Bay/Fenway
10 Sat Dec 31 00:31:00 2016 Eliot Hotel
11 Sat Dec 31 00:49:40 2016 XV Beacon
12 Sat Dec 31 01:07:07 2016 Hilton Garden Inn Boston Logan Airport
13 Sat Dec 31 01:17:39 2016 Residence Inn Boston Downtown/Seaport
14 Sat Dec 31 01:24:30 2016 The Verb Hotel
15 Sat Dec 31 01:47:51 2016 Kimpton Nine Zero Hotel
16 Sat Dec 31 02:28:07 2016 The Godfrey Hotel Boston
17 Sat Dec 31 02:54:07 2016 The Boxer Boston
18 Sat Dec 31 03:07:55 2016 The Langham, Bost

In [15]:
fi.close()